# Create a Grafana Data Source and Dashboard

Follow the steps in this this tutorial to create a "stocks" Grafana dashboard using Iguazio data source <br>
Note that you need to create a Grafana service before running the notebook <br>
Creating a grafana service can be done from the "services" screen in the Platform

In [1]:
import nuclio

## Environment

In [2]:
%nuclio cmd -c pip install git+https://github.com/v3io/grafwiz --upgrade
%nuclio cmd -c pip install v3io_frames

## Function

In [3]:
# nuclio: start-code

In [4]:
from grafwiz import *
import os
import v3io_frames as v3f

In [5]:
def deploy_dashboard(context, 
                     grafana_url:str = 'http://grafana', 
                     streamview_url:str = 'http://nuclio-stocks-stream-viewer:8080',
                     v3io_container:str = 'users/' + os.getenv('V3IO_USERNAME'),
                     stocks_kv_table:str = 'stocks/stocks_kv',
                     stocks_tsdb_table:str = 'stocks/stocks_tsdb'):
    # Create datasources
    DataSource(name='Iguazio').deploy(grafana_url, use_auth=True)
    DataSource(name='stream-viewer', frames_url=streamview_url).deploy(grafana_url, use_auth=False, overwrite=False)
    
    # Verify the KV table can be shown
    client = v3f.Client('framesd:8081', container=v3io_container)
    client.execute(backend='kv', table=stocks_kv_table, command='infer')
    
    # Create grafana dashboard
    dash = Dashboard("stocks", start='now-7d', dataSource='Iguazio')

    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=stocks/stocks_kv;backend=kv;container=users/" + os.getenv('V3IO_USERNAME'))

    # Create a table and log viewer in one row
    tbl = Table('Current Stocks Value', span=12).source(table=stocks_kv_table,fields=['symbol','volume', 'price', 'sentiment', 'last_updated'],container=v3io_container)
    dash.row([tbl])

    # Create 2 charts on the second row
    metrics_row = [Graph(metric).series(table=stocks_tsdb_table, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']]
    metrics_row.append(Graph('sentiment').series(table=stocks_tsdb_table, fields=['sentiment'], filter='symbol=="$SYMBOL"', container=v3io_container))
    dash.row(metrics_row)
    
    # Create log veiwer panel
    log = Table('Articles Log', dataSource='stream-viewer', span=12)
    dash.row([log])
    
    # Deploy to Grafana
    return dash.deploy(grafana_url)

In [6]:
# nuclio: end-code

## Local test

In [7]:
from mlrun import code_to_function, run_local

fn = code_to_function('grafana_dashboard_deployer',
                      kind='job',
                      handler='deploy_dashboard',
                      project='stocks')
fn.spec.build.base_image = 'mlrun/mlrun'
fn.export('04-grafana.yaml')

> 2021-03-22 10:09:47,084 [info] function spec saved to path: 04-grafana.yaml


In [8]:
run_local(handler=deploy_dashboard)

> 2021-03-22 10:09:49,019 [info] starting run mlrun-dd8d2c-deploy_dashboard uid=2a5f287071a84818b9af2c2cf115f5cb DB=http://mlrun-api:8080
Datasource Iguazio already exists
Datasource Iguazio created successfully
Datasource stream-viewer already exists
Datasource stream-viewer created successfully
Dashboard stocks created successfully


AttributeError: 'Series' object has no attribute '_mgr'